In [1]:
from nodes import Filescan, Selection, Projection

In [2]:
# SELECT title, genres
# FROM movies 
# WHERE genres LIKE '%War%' 
#     and genres LIKE %Sci-Fi%

In [3]:
q = Filescan('ml-20m/movies.csv')
q = Selection('genres', 'CONTAINS', 'War', child=q)
q = Projection('title', 'genres', child=q)
q = Selection('genres', 'CONTAINS', 'Sci-Fi', child=q)

In [4]:
for result in q:
    print(result)

('Soldier (1998)', 'Action|Sci-Fi|War')
('Wings of Honneamise (Ôritsu uchûgun Oneamisu no tsubasa) (1987)', 'Animation|Drama|Sci-Fi|War')
('Slaughterhouse-Five (1972)', 'Comedy|Drama|Sci-Fi|War')
('Captain America (1990)', 'Action|Fantasy|Sci-Fi|Thriller|War')
('Macross Plus (1994)', 'Action|Drama|Sci-Fi|War')
('Battlestar Galactica (2003)', 'Drama|Sci-Fi|War')
('Starship Troopers 2: Hero of the Federation (2004)', 'Action|Horror|Sci-Fi|War')
('Wizards (1977)', 'Animation|Fantasy|Sci-Fi|War')
('Threads (1984)', 'Drama|Sci-Fi|War')
('Black Moon (1975)', 'Fantasy|Mystery|Sci-Fi|War')
('Starship Troopers 3: Marauder (2008)', 'Action|Sci-Fi|War')
('MGS: Philanthropy (2009)', 'Action|Adventure|Sci-Fi|War')
('Invisible Agent (1942)', 'Adventure|Comedy|Sci-Fi|Thriller|War')
('Battle: Los Angeles (2011)', 'Action|Sci-Fi|War')
('X-Men: First Class (2011)', 'Action|Adventure|Sci-Fi|Thriller|War')
('Transformers: Dark of the Moon (2011)', 'Action|Adventure|Sci-Fi|War|IMAX')
('Captain America: The